In [2]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcol
import data_processing, evaluate, losses, model, train

/space/hall5/sitestore/eccc/crd/ccrn/users/rpg002/miniconda3/envs/rsaenv_neurips/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import sys
import os
from data_processing import prepare_data
from train import train_model
from train import make_snapshot_data
from evaluate import evaluate_model
import torch
from pathlib import Path
########### If you have torch_geometric ##############
# from torch_geometric.loader import DataLoader
######## else maske manual torch dataset ##########
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
###############################################
from datetime import datetime
from argparse import ArgumentParser

In [9]:
output_dir=Path("/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/StatDownOc/output")
data_dir=Path("/fs/site5/eccc/crd/ccrn/users/rpg002/stat_downscaling-workshop/data")
work_dir=Path(output_dir,"work")

In [10]:
work_dir

PosixPath('/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/StatDownOc/output/work')

In [11]:
    train_data, val_data, test_data, stations, depths = prepare_data(
        data_dir=data_dir,
        work_dir=work_dir,
        year_range=(1999, 2000),
        stations=["P22", "P23", "P24", "P25", "P26"],
        target_variable="Temperature",
        train_ratio=0.7, ##Changed
        val_ratio=0.15  ##Changed
    )

<xarray.Dataset>
Dimensions:      (time: 132, station: 24, depth: 4301, lat: 24, lon: 24,
                  distance: 24)
Coordinates:
  * time         (time) datetime64[ns] 1999-02-10T00:06:14 ... 2000-09-15T08:...
  * station      (station) <U3 'P4' 'P5' 'P6' 'P7' ... 'P24' 'P25' 'P35' 'P26'
  * lat          (lat) float64 48.65 48.69 48.74 48.78 ... 49.84 50.0 50.0 50.0
  * lon          (lon) float64 -126.7 -127.2 -127.7 ... -143.6 -144.3 -145.0
  * distance     (distance) float64 0.0 3.694e+04 ... 5.001e+04 4.973e+04
  * depth        (depth) float64 0.5 1.5 2.5 ... 4.302e+03 4.304e+03 4.306e+03
Data variables:
    Temperature  (time, station, depth) float64 nan 8.411 8.403 ... nan nan nan
    Salinity     (time, station, depth) float64 nan 31.87 31.9 ... nan nan nan
    Oxygen       (time, station, depth) float64 nan nan nan nan ... nan nan nan
    Latitude     (time, station, depth) float64 nan 48.65 48.65 ... nan nan nan
    Longitude    (time, station, depth) float64 nan -126.7 -

In [12]:
train_data

(array([[[[-1.2439431 , -1.14598608, -1.01400363, -0.92521602],
          [-0.81337255, -0.66165733, -0.46569821, -0.30219042],
          [-0.17639558, -0.09848829,  0.00560165,  0.138687  ],
          [ 0.1910181 ,  0.3559435 ,  0.4466413 ,  0.5572542 ],
          [ 0.74076378,  0.92388493,  1.13834894,  1.37977982]],
 
         [[ 0.        ,  0.        ,  0.        ,  0.        ],
          [ 0.        ,  0.        ,  0.        ,  0.        ],
          [ 0.        ,  0.        ,  0.        ,  0.        ],
          [ 0.        ,  0.        ,  0.        ,  0.        ],
          [ 0.        ,  0.        ,  0.        ,  0.        ]],
 
         [[ 0.        ,  0.33333333,  0.66666667,  1.        ],
          [ 0.        ,  0.33333333,  0.66666667,  1.        ],
          [ 0.        ,  0.33333333,  0.66666667,  1.        ],
          [ 0.        ,  0.33333333,  0.66666667,  1.        ],
          [ 0.        ,  0.33333333,  0.66666667,  1.        ]]],
 
 
        [[[-1.25923765, -1.0